In [2]:
import requests

url = "https://coinmarketcap.com/fr/"
response = requests.get(url, timeout=20)
response.raise_for_status()

# Enregistrer le code HTML brut
with open("coinmarket.html", "w", encoding="utf-8") as f:
    f.write(response.text)

print("✅ Code HTML enregistré sous techsport.html")


✅ Code HTML enregistré sous techsport.html


In [3]:
from bs4 import BeautifulSoup
import re

# --- ⚠️ REMPLACER PAR LE CONTENU HTML RÉEL DE LA PAGE ⚠️ ---
# Ce code utilise une chaîne simple à titre d'illustration.
html_content = """
<a href="/currencies/hyperliquid/" class="cmc-link Event">
    <p class="sc-65e7f566-0 iPBTJf coin-item-name">Hyperliquid</p>
</a>
<td style="text-align:end">
    <div class="sc-fa25c04c-0 eAphWs rise">
        <span>96 823,75 €</span>
    </div>
</td>
<td style="text-align:end">
    <p color="text" style="white-space:nowrap" font-size="1" data-sensors-click="true" class="sc-71024e3e-0 bbHOdE">
        <span data-nosnippet="true" class="sc-11478e5d-1 jfWGhx">€418,276,079,476</span>
    </p>
</td>
"""

def extraire_donnees_crypto(html_content):
    """
    Extrait le nom, le prix et le cap. boursier des cryptomonnaies
    à partir du contenu HTML.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    cryptomonnaies = []

    # Le conteneur principal qui encapsule TOUTES les données d'une ligne de crypto
    # (Doit être ajusté pour cibler la ligne entière du tableau, par exemple un <tr>)
    # Dans cet exemple, nous allons simplement chercher les noms comme points d'ancrage.
    
    noms = soup.find_all('p', class_='coin-item-name')

    for nom_tag in noms:
        # 1. NOM DE LA CRYPTO
        nom = nom_tag.get_text(strip=True)
        
        # Le reste des données n'est pas directement sous 'nom_tag'.
        # Nous devons remonter à l'ancêtre commun (généralement la balise <tr> de la ligne).
        
        # Remonter à la ligne <tr> (à ajuster selon la structure réelle de la page)
        try:
            ligne_crypto = nom_tag.find_parent('tr') 
            if not ligne_crypto:
                # Si la structure n'est pas un tableau <tr>, remonter au conteneur principal
                ligne_crypto = nom_tag.find_parent('div', class_=re.compile(r'sc-[\w\d]+'))
            
            if not ligne_crypto: continue # Passer si l'élément parent n'est pas trouvé
            
            # 2. PRIX
            # Le prix se trouve dans le <span> qui est l'enfant du <div> avec la classe '.rise'
            prix_tag = ligne_crypto.select_one('div.rise > span') 
            prix = prix_tag.get_text(strip=True) if prix_tag else "N/A"

            # 3. CAP. BOURSIER (Market Cap)
            # Le cap boursier se trouve dans le <span> avec l'attribut data-nosnippet
            cap_tag = ligne_crypto.select_one('span[data-nosnippet="true"]')
            market_cap = cap_tag.get_text(strip=True) if cap_tag else "N/A"

            # Ajouter les données au dictionnaire
            cryptomonnaies.append({
                "nom": nom,
                "prix": prix,
                "market_cap": market_cap
            })

        except Exception as e:
            print(f"Erreur lors de l'extraction des données pour {nom}: {e}")
            continue

    return cryptomonnaies

# --- EXÉCUTION ---
cryptomonnaies = extraire_donnees_crypto(html_content)

if cryptomonnaies:
    print(f"Extraction réussie de {len(cryptomonnaies)} cryptomonnaies :\n")
    for crypto in cryptomonnaies:
        print(f"Nom: {crypto['nom']}, Prix: {crypto['prix']}, Market Cap: {crypto['market_cap']}")
else:
    print("Aucune donnée de crypto n'a pu être extraite.")

Aucune donnée de crypto n'a pu être extraite.


In [7]:
from bs4 import BeautifulSoup

# --- Contenu HTML Simulé (Structure de plusieurs lignes) ---
# NOTE : La structure réelle de la page est essentielle. Ici, on simule 
# que chaque ligne est une balise <tr> qui contient toutes les <td>.
html_content_generique = """
<table>
    <tr class="cmc-table-row">
        <td>
            <p class="sc-65e7f566-0 iPBTJf coin-item-name">Bitcoin</p>
        </td>
        <td style="text-align:end">
            <div class="sc-fa25c04c-0 eAphWs rise">
                <span>60 123,45 €</span>
            </div>
        </td>
        <td style="text-align:end">
            <p><span data-nosnippet="true" class="sc-11478e5d-1 jfWGhx">€1,185,200,000,000</span></p>
        </td>
    </tr>
    <tr class="cmc-table-row">
        <td>
            <p class="sc-65e7f566-0 iPBTJf coin-item-name">Ethereum</p>
        </td>
        <td style="text-align:end">
            <div class="sc-fa25c04c-0 eAphWs rise">
                <span>3 500,00 €</span>
            </div>
        </td>
        <td style="text-align:end">
            <p><span data-nosnippet="true" class="sc-11478e5d-1 jfWGhx">€420,000,000,000</span></p>
        </td>
    </tr>
</table>
"""

def extraire_toutes_les_cryptos(html_content):
    """
    Extrait les données de toutes les lignes de crypto.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    cryptomonnaies = []

    # ⚠️ ÉTAPE CRUCIALE : Ciblez le conteneur qui représente une ligne de données complète
    # Sur CoinMarketCap, il pourrait s'agir d'une <tr> avec une classe ou d'une <div> spécifique.
    # Dans notre simulation :
    lignes_crypto = soup.find_all('tr', class_='cmc-table-row') 
    
    if not lignes_crypto:
        print("Avertissement : Le sélecteur de ligne ('tr.cmc-table-row') n'a trouvé aucune ligne. Veuillez l'ajuster.")
        return []

    for ligne in lignes_crypto:
        crypto = {}
        
        # 1. NOM DE LA CRYPTO (Recherche DANS la ligne)
        nom_tag = ligne.find('p', class_='coin-item-name')
        crypto['nom'] = nom_tag.get_text(strip=True) if nom_tag else "N/A"

        # 2. PRIX (Recherche DANS la ligne)
        # SÉLECTEUR : <span> à l'intérieur d'une <div> ayant la classe '.rise'
        prix_tag = ligne.select_one('div.rise > span')
        crypto['prix'] = prix_tag.get_text(strip=True).replace(' ', '') if prix_tag else "N/A"

        # 3. CAP. BOURSIER (Market Cap) (Recherche DANS la ligne)
        # SÉLECTEUR : <span> ayant l'attribut 'data-nosnippet="true"'
        cap_tag = ligne.select_one('span[data-nosnippet="true"]')
        crypto['market_cap'] = cap_tag.get_text(strip=True) if cap_tag else "N/A"
        
        # Si au moins le nom est trouvé, on ajoute la ligne
        if crypto['nom'] != "N/A":
            cryptomonnaies.append(crypto)

    return cryptomonnaies

# --- EXÉCUTION ---
cryptomonnaies = extraire_toutes_les_cryptos(html_content_generique)

if cryptomonnaies:
    print(f"Extraction réussie de {len(cryptomonnaies)} cryptomonnaies (y compris Bitcoin) :\n")
    for crypto in cryptomonnaies:
        print(f"Nom: {crypto['nom']}, Prix: {crypto['prix']}, Market Cap: {crypto['market_cap']}")
else:
    print("Aucune donnée de crypto n'a pu être extraite.")

Extraction réussie de 2 cryptomonnaies (y compris Bitcoin) :

Nom: Bitcoin, Prix: 60123,45€, Market Cap: €1,185,200,000,000
Nom: Ethereum, Prix: 3500,00€, Market Cap: €420,000,000,000


In [8]:
from bs4 import BeautifulSoup

# --- Contenu HTML Simulé (Structure Générique) ---
# Ce HTML est un exemple. Remplacez-le par le contenu HTML réel de la page.
html_content_generique = """
<table>
    <tr class="cmc-table-row">
        <td><p class="sc-65e7f566-0 iPBTJf coin-item-name">Bitcoin</p></td>
        <td style="text-align:end"><div class="sc-fa25c04c-0 eAphWs rise"><span>60 123,45 €</span></div></td>
        <td style="text-align:end"><p><span data-nosnippet="true" class="sc-11478e5d-1 jfWGhx">€1,185,200,000,000</span></p></td>
    </tr>
    <tr class="cmc-table-row">
        <td><p class="sc-65e7f566-0 iPBTJf coin-item-name">Ethereum</p></td>
        <td style="text-align:end"><div class="sc-fa25c04c-0 eAphWs rise"><span>3 500,00 €</span></div></td>
        <td style="text-align:end"><p><span data-nosnippet="true" class="sc-11478e5d-1 jfWGhx">€420,000,000,000</span></p></td>
    </tr>
    <tr class="cmc-table-row">
        <td><p class="sc-65e7f566-0 iPBTJf coin-item-name">BNB</p></td>
        <td style="text-align:end"><div class="sc-fa25c04c-0 eAphWs rise"><span>550,00 €</span></div></td>
        <td style="text-align:end"><p><span data-nosnippet="true" class="sc-11478e5d-1 jfWGhx">€85,000,000,000</span></p></td>
    </tr>
    </table>
"""

def extraire_dix_premieres_cryptos(html_content):
    """
    Extrait les données (nom, prix, market_cap) des 10 premières cryptos trouvées.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    cryptomonnaies = []
    
    # ⚠️ ÉTAPE CRUCIALE : Ciblez le conteneur qui représente une ligne de données complète.
    # Dans la simulation, on utilise 'tr.cmc-table-row'. Vous DEVEZ vérifier
    # cette classe/balise sur la page réelle.
    lignes_crypto = soup.find_all('tr', class_='cmc-table-row') 
    
    if not lignes_crypto:
        print("Avertissement : Le sélecteur de ligne n'a trouvé aucune ligne. Vérifiez votre HTML et le sélecteur.")
        return []

    # Limiter l'itération aux 10 premières lignes
    for ligne in lignes_crypto[:10]:
        crypto = {}
        
        # 1. NOM DE LA CRYPTO (Recherche DANS la ligne)
        nom_tag = ligne.find('p', class_='coin-item-name')
        crypto['nom'] = nom_tag.get_text(strip=True) if nom_tag else "N/A"

        # 2. PRIX (Recherche DANS la ligne)
        prix_tag = ligne.select_one('div.rise > span')
        crypto['prix'] = prix_tag.get_text(strip=True).replace(' ', '') if prix_tag else "N/A"

        # 3. CAP. BOURSIER (Market Cap) (Recherche DANS la ligne)
        cap_tag = ligne.select_one('span[data-nosnippet="true"]')
        crypto['market_cap'] = cap_tag.get_text(strip=True) if cap_tag else "N/A"
        
        if crypto['nom'] != "N/A":
            cryptomonnaies.append(crypto)

    return cryptomonnaies

# --- EXÉCUTION ---
cryptomonnaies = extraire_dix_premieres_cryptos(html_content_generique)

if cryptomonnaies:
    print(f"Extraction réussie des {len(cryptomonnaies)} premières cryptomonnaies :\n")
    print("--------------------------------------------------------------------------------------------------")
    for i, crypto in enumerate(cryptomonnaies):
        print(f"N°{i+1:<2} | Nom: {crypto['nom']:<15} | Prix: {crypto['prix']:<10} | Market Cap: {crypto['market_cap']}")
    print("--------------------------------------------------------------------------------------------------")
else:
    print("Aucune donnée de crypto n'a pu être extraite.")

Extraction réussie des 3 premières cryptomonnaies :

--------------------------------------------------------------------------------------------------
N°1  | Nom: Bitcoin         | Prix: 60123,45€  | Market Cap: €1,185,200,000,000
N°2  | Nom: Ethereum        | Prix: 3500,00€   | Market Cap: €420,000,000,000
N°3  | Nom: BNB             | Prix: 550,00€    | Market Cap: €85,000,000,000
--------------------------------------------------------------------------------------------------


In [13]:
import requests
from bs4 import BeautifulSoup
# ... autres imports ...

# 1. Étape de Récupération (À faire sur votre machine)
URL = 'https://coinmarketcap.com/...' # Remplacer par l'URL réelle
try:
    reponse = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0'})
    # Stockage "temporaire" en mémoire (plus efficace que dans un fichier)
    html_content_reel = reponse.text 

    # 2. Étape d'Analyse (Parsing)
    cryptomonnaies = extraire_dix_premieres_cryptos(html_content_reel)
    # ... suite du code ...

except Exception as e:
    print(f"Erreur lors de la récupération de la page : {e}")

def extraire_dix_premieres_cryptos(html_content):
    """
    Parse le contenu HTML et extrait les données des 10 premières cryptos.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    cryptomonnaies = []
    
    # ⚠️ Sélecteur de Ligne : À ajuster si la structure du site change.
    lignes_crypto = soup.find_all('tr', class_='cmc-table-row') 
    
    if not lignes_crypto:
        print("Avertissement : Le sélecteur de ligne ('tr.cmc-table-row') n'a trouvé aucune ligne. Vérifiez votre HTML.")
        return []

    # Limiter l'itération aux 10 premières lignes (indices 0 à 9)
    for ligne in lignes_crypto[:10]:
        crypto = {}
        
        # 1. NOM DE LA CRYPTO (Classe: coin-item-name)
        nom_tag = ligne.find('p', class_='coin-item-name')
        crypto['nom'] = nom_tag.get_text(strip=True) if nom_tag else "N/A"

        # 2. PRIX (Sélecteur: div.rise > span)
        prix_tag = ligne.select_one('div.rise > span')
        # Nettoyage des espaces pour le prix
        crypto['prix'] = prix_tag.get_text(strip=True).replace(' ', '') if prix_tag else "N/A"

        # 3. CAP. BOURSIER (Market Cap) (Sélecteur: span[data-nosnippet="true"])
        cap_tag = ligne.select_one('span[data-nosnippet="true"]')
        crypto['market_cap'] = cap_tag.get_text(strip=True) if cap_tag else "N/A"
        
        if crypto['nom'] != "N/A":
            cryptomonnaies.append(crypto)

    return cryptomonnaies

# --- EXÉCUTION ---
cryptomonnaies = extraire_dix_premieres_cryptos(html_content_reel_simule)

if cryptomonnaies:
    print(f"Extraction réussie des {len(cryptomonnaies)} premières cryptomonnaies.\n")
    # Affichage au format JSON (demandé)
    print(json.dumps(cryptomonnaies, indent=4, ensure_ascii=False))
else:
    print("Échec de l'extraction des données.")

Avertissement : Le sélecteur de ligne ('tr.cmc-table-row') n'a trouvé aucune ligne. Vérifiez votre HTML.
Extraction réussie des 10 premières cryptomonnaies.

[
    {
        "nom": "Bitcoin",
        "prix": "60123,45€",
        "market_cap": "€1,185,200,000,000"
    },
    {
        "nom": "Ethereum",
        "prix": "3500,00€",
        "market_cap": "€420,000,000,000"
    },
    {
        "nom": "BNB",
        "prix": "550,00€",
        "market_cap": "€85,000,000,000"
    },
    {
        "nom": "Solana",
        "prix": "140,25€",
        "market_cap": "€65,100,000,000"
    },
    {
        "nom": "Ripple",
        "prix": "0,45€",
        "market_cap": "€24,500,000,000"
    },
    {
        "nom": "Cardano",
        "prix": "0,38€",
        "market_cap": "€14,000,000,000"
    },
    {
        "nom": "Dogecoin",
        "prix": "0,12€",
        "market_cap": "€18,000,000,000"
    },
    {
        "nom": "Avalanche",
        "prix": "30,50€",
        "market_cap": "€12,500,000,000"
 

In [14]:
import requests
from bs4 import BeautifulSoup
import json

# URL de la page des cryptomonnaies (remplacez par l'URL exacte)
URL = 'L_ADRESSE_URL_REELLE_DE_LA_PAGE_DU_TABLEAU'

def get_html_content(url):
    """Télécharge le contenu HTML de la page."""
    try:
        # Il est important d'utiliser un User-Agent pour simuler un vrai navigateur
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        reponse = requests.get(url, headers=headers, timeout=10)
        reponse.raise_for_status() # Lève une exception pour les codes d'erreur HTTP
        return reponse.text
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de l'URL {url}: {e}")
        return None

# --- NOUVEAU FLUX D'EXÉCUTION ---
# 1. TÉLÉCHARGEMENT
html_content_reel = get_html_content(URL)

if html_content_reel:
    # 2. PARSING (Le reste de votre fonction extraire_dix_premieres_cryptos reste le même)
    cryptomonnaies = extraire_dix_premieres_cryptos(html_content_reel)
    
    if cryptomonnaies:
        print(f"Extraction réussie des {len(cryptomonnaies)} premières cryptomonnaies.\n")
        print(json.dumps(cryptomonnaies, indent=4, ensure_ascii=False))
    else:
        print("Échec du parsing des données. Vérifiez les sélecteurs.")
else:
    print("Impossible de procéder au scraping sans le contenu HTML.")

Erreur lors de la récupération de l'URL L_ADRESSE_URL_REELLE_DE_LA_PAGE_DU_TABLEAU: Invalid URL 'L_ADRESSE_URL_REELLE_DE_LA_PAGE_DU_TABLEAU': No scheme supplied. Perhaps you meant https://L_ADRESSE_URL_REELLE_DE_LA_PAGE_DU_TABLEAU?
Impossible de procéder au scraping sans le contenu HTML.


In [15]:
!pip install pyppeteer


  Preparing metadata (setup.py) ... done
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-macosx_11_0_arm64.whl size=97748 sha256=a36923610d24556518cb56eabceb38b31feccc622c4310fdec8b5ccf6790c751
  Stored in directory: /Users/philippe/Library/Caches/pip/wheels/80/cf/6d/5d7e4c920cb41925a178b2d2621889c520d648bab487b1d7fd
Successfully built websockets
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: pyee
    Found existing installation: pyee 13.0.0
    Uninstalling pyee-13.0.0:
      Successfully uninstalled pyee-13.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
playwright 1.55.0 requires pyee<14,>=13, but you have pyee 11.1.1 which is incompatible.


In [17]:
# ... (votre code Pyppeteer et les fonctions 'get_html_content_pyppeteer' et 'extraire_dix_premieres_cryptos') ...

# --- Fonction principale (définition) ---
async def main():
    # ⚠️ REMPLACER PAR L'URL RÉELLE DE LA PAGE CRYPTO
    URL = 'L_ADRESSE_URL_REELLE_DE_LA_PAGE_DU_TABLEAU' 
    
    # 1. TÉLÉCHARGEMENT AVEC JS
    html_content_reel = await get_html_content_pyppeteer(URL)
    
    if html_content_reel:
        # 2. PARSING
        cryptomonnaies = extraire_dix_premieres_cryptos(html_content_reel)
        
        if cryptomonnaies:
            print("\n✅ Extraction réussie des 10 premières cryptomonnaies (après exécution JS) :")
            print(json.dumps(cryptomonnaies, indent=4, ensure_ascii=False))
        else:
            print("\n❌ Échec du parsing des données. Vérifiez les sélecteurs et le temps d'attente.")
    else:
        print("\n❌ Impossible de récupérer le contenu HTML via Pyppeteer.")

# --- NOUVELLE CELLULE D'EXÉCUTION ---
# Exécutez ceci dans une nouvelle cellule (après la définition de main)
await main()

[INFO] Starting Chromium download.


Lancement du navigateur Chromium (Pyppeteer)...


100%|██████████| 141M/141M [00:05<00:00, 24.2Mb/s] 
[INFO] Beginning extraction
[INFO] Chromium extracted to: /Users/philippe/Library/Application Support/pyppeteer/local-chromium/1181205


Erreur Pyppeteer : Browser closed unexpectedly:


❌ Impossible de récupérer le contenu HTML via Pyppeteer.
